# Part III: Model Training for Left Right Fingerprints

Now that we understand our data and have it normalized, we can begin experimenting with a model that will predict if a fingerprint originated from a left or right hand. Is this simple, yes. This will allow us to get into production faster and improve the baseline.

## Setup

Import TensorFlow and other necessary libraries:

In [ ]:
! pip install -r ../requirements.txt -q

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
import cv2

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from IPython.display import clear_output

## Check TF and Cuda versions

For reference [see this chart](https://www.tensorflow.org/install/source#gpu)

Expected output
```
TensorFlow: 2.14.0
TensorFlow Datasets: 4.9.3
OpenCV version: 4.8.1
Cuda version: 11.8
cudnn version: 8
```

In [ ]:
# check versions for Tensorflow and Cuda
print('TensorFlow: ' + tf.__version__)
print('TensorFlow Datasets: ' + tfds.__version__)
print('OpenCV version: ' + cv2.__version__)

# Check CUDA Version
sys_details = tf.sysconfig.get_build_info()
cuda_version = sys_details["cuda_version"]
print('Cuda version: ' + cuda_version)

# Check CUDNN Version
cudnn_version = sys_details["cudnn_version"]  
print('cudnn version: ' + cudnn_version)

In [ ]:
# scratch directory is apart of the .gitignore to ensure it is not committed to git
%env SCRATCH=../scratch
! [ -e "${SCRATCH}" ] || mkdir -p "${SCRATCH}"

scratch_path = os.environ.get('SCRATCH', './scratch')

# Create the training and validation dataset

This function streamlines the creation of TensorFlow datasets for training and validation, encapsulating several steps such as loading, preprocessing, caching, and prefetching, making it easier to manage datasets for machine learning models.

### Function's Main Operations:

- image_dataset_from_directory: Utilizes TensorFlow's image_dataset_from_directory function to create a dataset from the specified directory with provided configurations.
Data Transformations:
- cache: Caches the dataset in memory after the initial data loading for faster access during training.
- shuffle: Shuffles the dataset with a buffer size of 1000.
- prefetch: Prefetches data, preparing subsequent batches while the model is executing computations on the current batch, optimizing data pipeline performance.

### Return:

Returns the processed dataset ready for training or validation.

In [ ]:
import tensorflow as tf

AUTOTUNE = tf.data.AUTOTUNE

def create_image_dataset(directory, color_mode="grayscale", validation_split=0.2,
                         subset=None, seed=123, image_size=(96, 96), batch_size=32):
    dataset = tf.keras.utils.image_dataset_from_directory(
        directory,
        color_mode=color_mode,
        validation_split=validation_split,
        subset=subset,
        seed=seed,
        image_size=image_size,
        batch_size=batch_size
    )
    
    dataset = dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    return dataset

In [ ]:
train_ds = create_image_dataset(
    scratch_path + '/processed/hand',
    subset="training",
)

In [ ]:
val_ds = create_image_dataset(
    scratch_path + '/processed/hand',
    subset="validation",
)

Here are the first nine images from the training dataset with their corresponding labels.

In [ ]:
# Get class names by iterating through train_ds and adding 'left' and 'right'
class_names = train_ds.class_names if hasattr(train_ds, 'class_names') else [str(label) for label in sorted(set(train_ds.map(lambda x, y: y).unbatch().as_numpy_iterator()))]
class_names += ['left', 'right']
class_names = sorted(set(class_names))  # Ensure uniqueness and sort

print(class_names)

In [ ]:
import matplotlib.pyplot as plt

label_mapping = {0: 'left', 1: 'right'}  # Define the mapping

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"), cmap='gray')
        # Replace label with corresponding class name using the mapping
        plt.title(label_mapping.get(labels[i].numpy(), 'Unknown'))
        plt.axis("off")
plt.show()

Pass the datasets to the Keras Model.fit method for training in later steps.

Use the image_batch tensor of  shape (32, 96, 96, 1), which splits batch of 32 images of shape 96x96x3 (the last dimension refers to color channels grayscale). The label_batch is a tensor of the shape (32,), these are corresponding labels to the 32 images.

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

## Distrubited Training Strategy

`tf.distribute.Strategy` is a TensorFlow API to distribute training across multiple GPUs, multiple machines, or TPUs. Using this API, you can distribute your existing models and training code with minimal code changes. [Types of strategies](https://www.tensorflow.org/guide/distributed_training).

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

## GPU Memory Allocation

The `TF_GPU_ALLOCATORvariable` enables the memory allocator using cudaMallocAsync available since CUDA 11.2. It has fewer fragmentation issues than the default BFC memory allocator. Note, this could become the default in the future.

This environment variable is unset by default. To enable the variable, run the following command:

In [ ]:
# cuda_malloc_async has fewer fragmentation issues than the default BFC memory allocator - https://docs.nvidia.com/deeplearning/frameworks/tensorflow-user-guide/index.html#tf_gpu_allocator
TF_FORCE_GPU_ALLOW_GROWTH=True

os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
print(os.getenv("TF_GPU_ALLOCATOR"))

## Create Model Function

This function encapsulates the creation and compilation of a CNN model for image classification tasks. It provides flexibility to specify the class names and input image shape, enabling the user to adapt the model for different classification problems and data formats.

### Function's Main Operations:
- Model Creation:
    - Utilizes a distributed strategy (tf.distribute.MirroredStrategy()) for training across multiple devices (if available).
    - Constructs a sequential model using Keras (Sequential).
    - The model architecture consists of several layers:
        - Convolutional layers (Conv2D) with varying filter sizes and activation functions (ReLU), followed by max-pooling layers (MaxPooling2D).
        - Flattening layer (Flatten) to transform the output of the convolutional layers into a 1D tensor.
        - Dense layers (Dense) for classification, with ReLU activation for hidden layers and linear activation for the output layer.

### Compilation:
- Compiles the model using the Adam optimizer ('adam'), sparse categorical cross-entropy loss function (tf.keras.losses.SparseCategoricalCrossentropy), and accuracy as the metric for evaluation.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential

def create_model(class_names=None, input_image_shape=(96, 96, 1)):
    if class_names is None:
        class_names = ['left', 'right',]  # Define default class names here

    strategy = tf.distribute.MirroredStrategy()
    num_classes = len(class_names)

    with strategy.scope():
        model = Sequential([
          layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=input_image_shape, name="model"),
          layers.MaxPooling2D(),
          layers.Conv2D(32, 3, padding='same', activation='relu'),
          layers.MaxPooling2D(),
          layers.Conv2D(64, 3, padding='same', activation='relu'),
          layers.MaxPooling2D(),
          layers.Flatten(),
          layers.Dense(128, activation='relu'),
          layers.Dense(num_classes)
        ])

        model.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])
    
    return model

In [ ]:
# build and compile the mode
model = create_model()

In [ ]:
model.summary()

## Training Function

This function streamlines the process of training a neural network model by encapsulating the training routine with a specific model, datasets, and epochs, allowing for easier experimentation and training within the notebook or Python environment.

In [ ]:
def train_model(model, train_data=train_ds, val_data=val_ds, epochs=10):
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=epochs
    )
    return history

In [ ]:
# Assuming you already have your 'model', 'train_ds', and 'val_ds' variables defined
trained_history = train_model(model, train_ds, val_ds, epochs=10)

The performance metrics  give insight into how well your model has learned from the training data and how well it generalizes to unseen validation data.

- Loss: This represents the error or discrepancy between the predicted and actual values. Lower values indicate better performance, as it implies that the model's predictions are closer to the actual values.
- Accuracy: This metric indicates the proportion of correctly classified samples out of the total samples. An accuracy of 0.9443 for training data means that the model correctly predicted the classes for approximately 94.43% of the training set.
- Val_loss (Validation Loss): This measures the error between the predicted values and the actual values on the validation dataset. Similar to training loss, lower values are better, indicating that the model generalizes well to unseen data.
- Val_accuracy (Validation Accuracy): This represents the proportion of correctly classified samples out of the total samples in the validation dataset. An accuracy of 0.8967 for validation data suggests that the model correctly predicted the classes for approximately 89.67% of the validation set.

Overall, a high training accuracy and a relatively close validation accuracy indicate that the model is performing well, but you may want to watch out for overfitting and fine-tune the model further if necessary. Regularization techniques, adjusting model complexity, or increasing data diversity might help improve generalization to unseen data.

## Performance Visualization Function

This function, plot_training_history, takes in the history object returned by the model's fit method and the epochs parameter. It then plots the training and validation accuracy as well as the training and validation loss in two subplots, providing insights into the model's performance during training.

In [ ]:
import matplotlib.pyplot as plt
import os
from datetime import datetime

def plot_training_history(history, epochs, save_dir=None):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    
    # Save the plot to a directory if specified
    if save_dir:
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        
        current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        plot_filename = f"training_history_plot_{current_time}.png"
        plot_path = os.path.join(save_dir, plot_filename)
        
        plt.savefig(plot_path)
        print(f"Plot saved to: {plot_path}")
    
    # Show the plot
    plt.show()

In [ ]:
# Execute the performance visualization
plot_training_history(history=trained_history, epochs=10, save_dir='../reports/')

# Serialize Model

In [ ]:
model_version = os.environ.get('VERSION', "1")

model_path = scratch_path + "/models/hand/" + model_version + "/model.savedmodel"
model.save(model_path)

# Test with a sample

## Load the trained model

In [ ]:
model_path = scratch_path + "/models/hand/" + model_version + "/model.savedmodel"
model = keras.models.load_model(model_path)

## Load a random sample input image

In [ ]:
# select random images
import random
import matplotlib.pyplot as plt

samples_path = scratch_path + "/processed/real/"
file_list = os.listdir(samples_path)
print('Files in path: ' + str(len(file_list)))

test_image = random.choice(file_list)
print('Selected: ' + test_image)

# loads an image into PIL format.
img = tf.keras.utils.load_img(
    samples_path + test_image,
    color_mode="grayscale",
    target_size=(96, 96),
    #interpolation='nearest',
)

plt.imshow(img, cmap='gray')

# converts a PIL image instance to a numpy array
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch
print("shape:",img_array.shape)

## Generate a prediction

In [ ]:
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

class_mapping = {0: 'left', 1: 'right'}  # Define your class mapping

predicted_label = class_mapping[np.argmax(score)]
confidence = 100 * np.max(score)

print(
    f"This image {test_image} is predicted as {predicted_label} with a {confidence:.2f}% confidence."
)

# Check if the predicted label is in the test image name (case insensitive)
is_label_in_image_name = predicted_label.lower() in test_image.lower()
print(f"Is predicted label in the image name? {is_label_in_image_name}")

# Upload to S3

## List the available buckets

In [ ]:
# list objects using the aws s3 cli
! aws s3 ls

## List the dynamically created bucket for the demo

In [ ]:
import boto3

s3_client = boto3.client('s3')
response = s3_client.list_buckets()

filtered_buckets=[]

for bucket in response['Buckets']:
    bucket_name = bucket['Name']
    if bucket_name.startswith('sagemaker-fingerprint-'):
        filtered_buckets.append(bucket_name)
    
print(filtered_buckets)

## Upload the model to the demo bucket

In [ ]:
import subprocess

for bucket in filtered_buckets:
    command = f"aws s3 sync ../scratch/models/hand s3://{bucket}/models/hand"
    subprocess.run(command, shell=True)

# Next Steps

Congrats! We have our first baseline model that predicts which hand an unknown fingerprint originated stored in AWS S3. Next:

Verify the model in AWS S3:
1. Go to your AWS Console
1. Go to S3 and navigate to the sagemaker-fingerprint-data- bucket
1. Go to the models subdirectory and step into the directories if you want

Check inference:
1. Go to your OpenShift Console
1. Navigate to the *Project* `models`
1. Under *Networking* go to *Routes*
1. Click the `gradio-client` URL
1. Interact with the model
